In [ ]:
import os
project_dir = "/Volumes/mnt/Workspace/ai-trading-agents"
os.chdir(project_dir)

In [ ]:
%reload_ext autoreload
%autoreload 2


In [ ]:
import dotenv
dotenv.load_dotenv()

In [ ]:
from langchain.agents import create_agent
from src import middleware
from src import tools
from src.context import Context, get_context
import json
from src import db
from src.models import get_model

In [ ]:
model_name = "deepseek"
langchain_model = get_model(model_name)

In [ ]:
run_id = "09404420-6ca9-4cd5-9f21-704d1b0fac0a"

In [ ]:
context = await get_context(run_id)

In [ ]:
user = context.bot.user
watchlist = context.bot.watchlist
portfolio = context.bot.portfolio

In [ ]:
user_name = "User Name: " + " ".join([user.firstName, user.lastName])

In [ ]:
watchlist = "Watchlist: " + \
    ", ".join([w.ticker for w in watchlist])

In [ ]:
from src.tools.actions import ListPositionsAct

In [ ]:
positions_markdown = await ListPositionsAct().arun(bot_id=context.bot.id)

In [ ]:
context.bot.DailyPortfolioSnapshot

In [ ]:
create_performance_analysis(context.bot.DailyPortfolioSnapshot)

In [ ]:
def create_performance_analysis(snapshots):
    """
    Create comprehensive performance analysis with key metrics
    """
    sorted_snapshots = sorted(snapshots, key=lambda x: x.date)

    if len(sorted_snapshots) < 2:
        return {"error": "Insufficient data for analysis"}

    # Extract values
    dates = [s.date.strftime('%Y-%m-%d') for s in sorted_snapshots]
    portfolio_values = [s.portfolioValue for s in sorted_snapshots]

    # Calculate returns
    daily_returns = []
    for i in range(1, len(portfolio_values)):
        daily_return = (
            (portfolio_values[i] - portfolio_values[i-1]) / portfolio_values[i-1]) * 100
        daily_returns.append(daily_return)

    # Key metrics
    total_return = portfolio_values[-1] - portfolio_values[0]
    total_return_pct = (total_return / portfolio_values[0]) * 100

    # Volatility (standard deviation of daily returns)
    volatility = np.std(daily_returns) if daily_returns else 0

    # Best and worst days
    best_day_pct = max(daily_returns) if daily_returns else 0
    worst_day_pct = min(daily_returns) if daily_returns else 0

    performance_analysis = {
        'summary': {
            'period': f"{dates[0]} to {dates[-1]}",
            'days': len(dates),
            'starting_value': portfolio_values[0],
            'ending_value': portfolio_values[-1],
            'total_return': total_return,
            'total_return_percent': total_return_pct,
            'average_daily_return': sum(daily_returns) / len(daily_returns) if daily_returns else 0,
            'volatility': volatility,
            'best_day_percent': best_day_pct,
            'worst_day_percent': worst_day_pct
        },
        'time_series': [
            {
                'date': date,
                'portfolio_value': value,
                'cumulative_return': ((value - portfolio_values[0]) / portfolio_values[0]) * 100
            }
            for date, value in zip(dates, portfolio_values)
        ],
        'daily_returns': [
            {
                'date': dates[i],
                'return_percent': daily_returns[i-1] if i > 0 else 0
            }
            for i in range(len(dates))
        ]
    }

    return performance_analysis

In [ ]:
create_performance_analysis(context.bot.DailyPortfolioSnapshot)